In [3]:
import numpy as np
from numpy import nan

from tabulate import tabulate
from itertools import zip_longest 
from collections import namedtuple
from stack import Stack
from actions import Moove, Action


In [ ]:
EPISODE = 2500
DECAY = 0.998
epsilon = 0.9
ACTION_REWARD = -1
FINISH_REWARD = 100
LEARNING_RATE = 0.1
DISCOUNT = 0.95
STACK_SIZE = 10


In [6]:
class Env:
    
    def __init__(self, size):
        self.A = Stack(size)
        self.B = Stack(0)
        self.moover = Moove(self.A, self.B)
        self.actions = Action(self.A, self.B)
        self.cum_reward = self.moover.reward
    
        
    def state(self):
        return (self.A.stack, self.B.stack)
    
    def reset(self, size):
        self.A = Stack(size)
        self.B = Stack(0)
        
    def actions_available(self):
        return self.actions.possible_actions()
    
    def reward(self):
        if self.B.stack.size == 0 and np.all(self.A.stack[:-1] <= self.A.stack[1:]):
             self.cum_reward += 100
        return (self.cum_reward)
        
    def take_action(self):
        
    
    def print_stack(self):
        print("-------------------")
        for a, b in zip_longest(reversed(self.A.stack), reversed(self.B.stack), fillvalue='xxx'):
          
          print(f"{a}\t{b}")
        print("-------------------")
  


IndentationError: expected an indented block (3424782164.py, line 28)

In [5]:
env = Env(10)

AttributeError: 'Env' object has no attribute 'cum_reward'

In [4]:
env.moover.swap_a()
env.print_stack()
print(env.actions_available())
print(env.moover.reward)

env.moover.push_b()
print(env.actions_available())
print(env.moover.reward)


env.moover.push_b()
print(env.actions_available())
print(env.moover.reward)


env.moover.push_b()
env.print_stack()
print(env.actions_available())
print(env.moover.reward)



-------------------
8	xxx
9	xxx
7	xxx
6	xxx
5	xxx
4	xxx
3	xxx
2	xxx
1	xxx
0	xxx
-------------------
['push_a', 'rotate_a', 'inverse_rotate_a', 'swap_a']
-1
['push_a', 'rotate_a', 'inverse_rotate_a', 'swap_a']
-2
['push_b', 'rotate_b', 'inverse_rotate_b', 'swap_b', 'push_a', 'rotate_a', 'inverse_rotate_a', 'swap_a', 'rotate', 'reverse', 'swap']
-3
-------------------
6	7.0
5	9.0
4	8.0
3	xxx
2	xxx
1	xxx
0	xxx
-------------------
['push_b', 'rotate_b', 'inverse_rotate_b', 'swap_b', 'push_a', 'rotate_a', 'inverse_rotate_a', 'swap_a', 'rotate', 'reverse', 'swap']
-4
